# InsertParameters
This class enables parameters to be inserted into a model from a copasi output file, pandas dataframe or a python dictionary. 

In [3]:
%matplotlib inline
import os,glob
import site
site.addsitedir('/home/b3053674/Documents/PyCoTools')
import PyCoTools
from PyCoTools.PyCoToolsTutorial import test_models
import os
import pandas
import logging
logging.basicConfig(format = '%(levelname)s:%(message)s')
LOG=logging.getLogger()

##this can be anywhere you like
tutorial_directory = '/home/b3053674/Documents/PyCoTools/PyCoTools/PyCoToolsTutorial'
## module containing copasi models in string format
TM = test_models.TestModels()
kholodenko_model_string = TM.get_kholodenko_variant1() ## published kholodenko model
kholodenko_model = os.path.join(tutorial_directory, 'kholodenko_model.cps')

with open(kholodenko_model, 'w') as f:
    f.write(kholodenko_model_string)

LOG.info('written kholodenko model to {}'.format(kholodenko_model))
LOG.info('Checking that kholodenko file exists: ... {}'.format(os.path.isfile(kholodenko_model)))

INFO:root:__init__:<module>:26:Initializing PyCoTools
INFO:root:__init__:<module>:27:Initializing logging System
INFO:root:__init__:<module>:28:logging config file at: /home/b3053674/Documents/PyCoTools/PyCoTools/logging_config.conf
INFO:root:<ipython-input-3-f58b6052b3b5>:<module>:23:written kholodenko model to /home/b3053674/Documents/PyCoTools/PyCoTools/PyCoToolsTutorial/kholodenko_model.cps
INFO:root:<ipython-input-3-f58b6052b3b5>:<module>:24:Checking that kholodenko file exists: ... True


Use the `parameter_dict` keyword to change model parameters from a python dictionary:

In [4]:
GMQ_before=PyCoTools.pycopi.GetModelQuantities(kholodenko_model)

## Here are our model quantities
print('(MAPKKK activation).K1 before: {}'.format(GMQ_before.get_all_model_variables()['(MAPKKK activation).K1']['value']))
    
    
##create a python dictionary dict[parameter_name]=value
parameter_dct={'(MAPKKK activation).K1':8123}
PyCoTools.pycopi.InsertParameters(kholodenko_model,parameter_dict = parameter_dct)

GMQ_after=PyCoTools.pycopi.GetModelQuantities(kholodenko_model)

## Here are our model quantities
print('(MAPKKK activation).K1 after: {}'.format(GMQ_after.get_all_model_variables()['(MAPKKK activation).K1']['value']))
    

(MAPKKK activation).K1 before: 10
(MAPKKK activation).K1 after: 10


Default behaviour when inserting metabolite values is to convert between particle numbers and concentrations automatically and insert both into the CopasiML. Change `CHECK_MODEL` variable to `True` to open the model and confirm for yourself.

In [8]:
import PyCoTools,os
GMQ_before=PyCoTools.pycopi.GetModelQuantities(kholodenko_model)
for key,val in  GMQ_before.get_metabolites().items():
    print (key,val)

    
metab_dct={'Mek1-PP':1000}
PyCoTools.pycopi.InsertParameters(kholodenko_model,ParameterDict=metab_dct)
print ('\n\n\n')
GMQ_after=PyCoTools.pycopi.GetModelQuantities(kholodenko_model)
for key,val in  GMQ_after.get_metabolites().items():
    if key == metab_dct.keys()[0]:
        print (key,val)
        
CHECK_MODEL=False
if CHECK_MODEL:
    os.system('CopasiUI {}'.format(kholodenko_model))


AttributeError: GetModelQuantities instance has no attribute 'get_metabolites'

## From Pandas
A `pandas.DataFrame` is a very useful python structure. Parameters are be imported directly from a DataFrame. 

In [ ]:
import PyCoTools,os
import pandas
GMQ_before=PyCoTools.pycopi.GetModelQuantities(kholodenko_model)
for key,val in  GMQ_before.get_all_model_variables().items():
    print key,val['value']

print '\n\n'
param_dct={'Mek1-P':1000,
           'Mek1-PP':0.001,
           '(phosphorylation of MAPKK).KK3':97}
df=pandas.DataFrame(param_dct,index=[0])
print df
PyCoTools.pycopi.InsertParameters(kholodenko_model,DF=df)
print '\n\n\n'
GMQ_after=PyCoTools.pycopi.GetModelQuantities(kholodenko_model)
for key,val in  GMQ_before.get_all_model_variables().items():
    print key,val['value']

        
CHECK_MODEL=False
if CHECK_MODEL:
    os.system('CopasiUI {}'.format(kholodenko_model))


# From File
To demonstrate InsertParameters from file, lets do a parameter estimation to some synthetic data and insert those parameters directly from file. 

In [2]:
import pandas,os,PyCoTools

GMQ_before=PyCoTools.pycopi.GetModelQuantities(kholodenko_model)
for key,val in  GMQ_before.get_all_model_variables().items():
    print key,val['value']


data_report='InsertParametersEstimationData.txt'
##Simulate some synthetic data
TC=PyCoTools.pycopi.TimeCourse(kholodenko_model,End=1000,StepSize=1,
                               Intervals=1000,
                               ReportName=data_report)

# setup parameter estimation with all default parameters
estimation_results='InsertParametersEstimationResults.txt'
PE=PyCoTools.pycopi.ParameterEstimation(kholodenko_model,data_report,
                                       ReportName = estimation_results)
PE.write_item_template()
PE.set_up()
## run
PE.run() 



INFO:root:__init__:18:Initializing PyCoTools
INFO:root:__init__:19:Initializing Logging System
INFO:root:__init__:20:logging config file at: /home/b3053674/anaconda2/lib/python2.7/site-packages/PyCoTools/logging_config.conf
(phosphorylation of MAPK-P).k8 111567.0
(dephosphorylation of MAPK-PP).KK9 0.271666
(dephosphorylation of MAPKK-P).KK6 0.00300255
(phosphorylation of MAPKK).KK3 93469.1
(MAPKKK activation).K1 1.17974e-05
Erk2-P 1.61940210447e+15
(dephosphorylation of MAPK-PP).V9 1.32781e-06
(MAPKKK inactivation).KK2 0.0012414
(MAPKKK activation).V1 27598.7
(phosphorylation of MAPK).k7 0.00150075
Mos 1442333069.41
(phosphorylation of MAPK).KK7 1.47402
(dephosphorylation of MAPK-P).KK10 19.5641
(phosphorylation of MAPK-P).KK8 0.00024886
Erk2-PP 5.42690727333e+15
(dephosphorylation of MAPKK-PP).V5 1.08283e-06
(MAPKKK inactivation).V2 55.0312
Mos-P 602214179.0
(dephosphorylation of MAPK-P).V10 1.04624e-06
(MAPKKK activation).Ki 33.4784
Mek1 2.81988595959e+12
Mek1-PP 90819920335.0
(phosp

<PyCoTools.pycopi.Run instance at 0x7f103d965ea8>

Now lets check that its worked: 

In [9]:
import PyCoTools
import pandas
## recall that best parameters are at the bottom of the copasi generated results file
print pandas.DataFrame.from_csv(estimation_results,sep='\t').tail() #so use tail to get end
I=PyCoTools.pycopi.InsertParameters(kholodenko_model,ParameterPath=estimation_results)


             [Erk2]_0   [Mos]_0  [Erk2-P]_0  [Erk2-PP]_0  [Mos-P]_0  [Mek1]_0  \
[Mek1-PP]_0                                                                     
0.000410     0.001318  0.000258     12.7708     0.001070   0.000009  0.004972   
0.000306     0.000846  0.000209     10.4456     0.001220   0.000002  0.004273   
0.000274     0.002432  0.000161     15.4971     0.000904   0.000002  0.004711   
0.000311     0.001536  0.000125     11.4934     0.001432   0.000003  0.004624   
0.000254     0.000797  0.000139     13.1313     0.001518   0.000010  0.004984   

             [Mek1-P]_0  (phosphorylation of MAPKK).KK3  \
[Mek1-PP]_0                                               
0.000410       0.000006                         32154.9   
0.000306       0.000003                         49571.2   
0.000274       0.000008                         67201.7   
0.000311       0.000006                         51680.3   
0.000254       0.000005                         29224.0   

             (MAPK

In [ ]:
import pandas,os,PyCoTools

GMQ_before=PyCoTools.pycopi.GetModelQuantities(kholodenko_model)
for key,val in  GMQ_before.get_all_model_variables().items():
    print key,val['value']


data_report='InsertParametersEstimationData.txt'
##Simulate some synthetic data
TC=PyCoTools.pycopi.TimeCourse(kholodenko_model,End=1000,StepSize=1,
                               Intervals=1000,
                               ReportName=data_report)

# setup parameter estimation with all default parameters
estimation_results='InsertParametersEstimationResults.txt'
PE=PyCoTools.pycopi.ParameterEstimation(kholodenko_model,data_report,
                                       ReportName = estimation_results)
PE.write_item_template()
PE.set_up()
PE.run()

    


Now lets check:

In [ ]:
import PyCoTools,pandas,os


print pandas.DataFrame.from_csv(estimation_results,sep='\t').iloc[-1]


## by default parameter sets are sorted by objective function value (lowest first) and index is set to 0
I=PyCoTools.pycopi.InsertParameters(kholodenko_model,ParameterPath=estimation_results,Index = 0)

GMQ_after = PyCoTools.pycopi.GetModelQuantities(kholodenko_model)

## Easiest way to check the parameters have been inserted is to look at the model in the GUI
os.system('CopasiUI {}'.format(kholodenko_model)) ##remember to close the GUI

# From Folder of Parameter Estimation Files
First lets create some parameter estimation data to work with using the `RunMultiplePEs`
class

In [6]:
import PyCoTools,os,pandas
output_directory = 'MultiplePEs'
RMPE=PyCoTools.pycopi.RunMultiplePEs(kholodenko_model,data_report,
                               CopyNumber=2,NumberOfPEs=3, #6 total
                               OutputDir = output_directory)
RMPE.write_config_template()
RMPE.set_up()
RMPE.run()

INFO:PyCoTools.pycopi:pycopi:4455:creating a directory for analysis in : 

/home/b3053674/Documents/PyCoTools/PyCoTools/PyCoToolsTutorial/MultiplePEs
INFO:PyCoTools.pycopi:pycopi:4314:writing PE config template for model: /home/b3053674/Documents/PyCoTools/PyCoTools/PyCoToolsTutorial/Kholodenko.cps
INFO:PyCoTools.pycopi:pycopi:4435:Copying copasi file 2 times
INFO:PyCoTools.pycopi:pycopi:4354:setting up scan for model : /home/b3053674/Documents/PyCoTools/PyCoTools/PyCoToolsTutorial/Kholodenko_0.cps
INFO:root:pycopi:3466:defining report
INFO:PyCoTools.pycopi:pycopi:4354:setting up scan for model : /home/b3053674/Documents/PyCoTools/PyCoTools/PyCoToolsTutorial/Kholodenko_1.cps
INFO:root:pycopi:3466:defining report
INFO:PyCoTools.pycopi:pycopi:4379:Setup Took 0.35977602005 seconds
INFO:PyCoTools.pycopi:pycopi:4362:<PyCoTools.pycopi.Scan instance at 0x7f103d9526c8>
INFO:PyCoTools.pycopi:pycopi:4302:Running model: 0
INFO:PyCoTools.pycopi:pycopi:4302:Running model: 1
INFO:PyCoTools.pycopi:py

The `RunMultiplePEs` class outputs parameter estimation data to a folder - one file per `CopyNumber`. We can insert parameters into a copasi model directly from the folder of estimation results. 

In [8]:
import PyCoTools,os,pandas

## Insert best parameter set (index 0)
I=PyCoTools.pycopi.InsertParameters(kholodenko_model,ParameterPath=output_directory,Index=0)

##Insert second best parameter set
I=PyCoTools.pycopi.InsertParameters(kholodenko_model,ParameterPath=output_directory,Index=1)

## ... and so on. 

INFO:PyCoTools.PEAnalysis:PEAnalysis:90:Parsing data from MultiplePEs into python
    Mek1-PP      Erk2       Mos    Erk2-P  Erk2-PP     Mos-P      Mek1  \
0  0.000012  0.000001  0.000001  0.000001  11.6950  0.000003  0.002358   
1  0.000004  0.000001  0.000001  0.000002  11.6792  0.000001  0.001973   
2  0.000003  0.001060  0.000195  0.000085  11.7685  0.000041  0.000001   

     Mek1-P  (phosphorylation of MAPKK).KK3  (MAPKKK activation).K1    ...     \
0  0.002308                        0.001140                0.000002    ...      
1  0.002698                     9363.740000                0.000001    ...      
2  0.004805                        0.650876            47016.400000    ...      

   (MAPKKK activation).V1  (MAPKKK activation).Ki  \
0               153.73400                 2.26860   
1               254.77000                 6.03164   
2                 9.96174            182761.00000   

   (phosphorylation of MAPK).KK7  (dephosphorylation of MAPK-P).KK10  \
0          